In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np

In [2]:
file_path = '../Sup/genres.csv'
dataset = pd.read_csv(file_path)

dataset.head()

,review,genre,sentiment
0,Confidently directed dark brooding and pack...,Action,Positive
1,Nolans have given the character this great man...,Action,Positive
2,there is action explosions and stunt work but...,Action,Positive
3,The acting story atmosphere and actions scenes...,Action,Positive
4,I recommend it to anyone who likes action movi...,Action,Positive


In [3]:
print(dataset['genre'].unique())
encording_dict = {'Action':0,'Horror':1,'Comedy':2,'Romance':3,'Animation':4}
dataset['categories'] = dataset['genre'].apply(lambda x:encording_dict[x])
from sklearn import preprocessing

['Action' 'Horror' 'Comedy' 'Romance' 'Animation']


In [4]:
#reviews = dataset['review'].apply(lambda x:BeautifulSoup(x,'lxml').text)

In [5]:
data = dataset.loc[:,['review','categories']]

In [6]:
from sklearn.model_selection import train_test_split
train,test = train_test_split(data,test_size=0.1,random_state=1)
X_train = train['review'].values
X_test = test['review'].values
#y_train = preprocessing.label_binarize(train['categories'].values, classes=[0, 1, 2, 3,4])
y_train = train['categories'].values
y_test = test['categories'].values
#y_test = preprocessing.label_binarize(test['categories'].values, classes=[0, 1, 2, 3,4])

In [7]:
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords

en_stopwords = set(stopwords.words("english"))

from sklearn.feature_extraction.text import CountVectorizer

vectorize = CountVectorizer(analyzer='word',
                            tokenizer=word_tokenize,
                            lowercase=True,
                            stop_words=en_stopwords)



In [8]:
from sklearn.model_selection import StratifiedKFold

kFold = StratifiedKFold(n_splits=5,shuffle=True,random_state=1)

In [9]:
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsClassifier

#pipeline_svm = make_pipeline(vectorize,
#                             KNeighborsClassifier(n_neighbors = 5,
#                                                  metric = 'minkowski',
#                                                  p=1))

from sklearn.svm import SVC

pipeline_svm = make_pipeline(vectorize,
                             SVC(probability=True,
                                 kernel='rbf',
                                 class_weight='balanced'))


In [10]:
from sklearn.model_selection import GridSearchCV

svm_grid = GridSearchCV(pipeline_svm,
                        cv=kFold,
                        param_grid={},
                        verbose=1,
                        n_jobs=1)

In [11]:
svm_grid.fit(X_train,y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\deela\.conda\envs\ML_env\lib\site-packages\sklearn\feature_extraction\text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
C:\Users\deela\.conda\envs\ML_env\lib\site-packages\sklearn\feature_extraction\text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
C:\Users\deela\.conda\envs\ML_env\lib\site-packages\sklearn\feature_extraction\text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop wor

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=1, shuffle=True),
             estimator=Pipeline(steps=[('countvectorizer',
                                        CountVectorizer(stop_words={'a',
                                                                    'about',
                                                                    'above',
                                                                    'after',
                                                                    'again',
                                                                    'against',
                                                                    'ain',
                                                                    'all', 'am',
                                                                    'an', 'and',
                                                                    'any',
                                                                    'are',
                   

In [12]:
svm_grid.score(X_test,y_test)

0.93

In [13]:
print(svm_grid.best_index_)
print(svm_grid.best_params_)
print(svm_grid.best_score_)

model = svm_grid.best_estimator_

0
{}
0.8911111111111112


In [18]:

from sklearn.metrics import make_scorer, accuracy_score, f1_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix, roc_auc_score, recall_score, precision_score

#pred_proba = model.predict_proba(X_test)[:,1]
pred = model.predict(X_test)

y = y_test
#auc = roc_auc_score(y, pred_proba)
acc = accuracy_score(y, pred)
#f1 = f1_score(y, pred)
#prec = precision_score(y, pred)
#rec = recall_score(y, pred)
#result = { 'acc': acc, 'precision': prec, 'recall': rec}
print(acc)


0.93


In [15]:
#import pickle
#with open('../Sup/gen_kmn_model1.0.sav','wb') as pf:
#    pickle.dump(svm_grid,pf)